In [1]:
# Mount google drive to this envrionment

from google.colab import drive
drive.mount('drive')

Drive already mounted at drive; to attempt to forcibly remount, call drive.mount("drive", force_remount=True).


In [5]:
# Read the tweeets file

import pandas as pd

tweet_data = pd.DataFrame()

dir = 'drive/My Drive/data/russ_twit/'

for i in range(1,10):
  temp = pd.read_csv(dir + 'IRAhandle_tweets_' + str(i) + '.csv')
  tweet_data = tweet_data.append(temp)
  
tweet_data.head()

,external_author_id,author,content,region,language,publish_date,harvested_date,following,followers,updates,post_type,account_type,new_june_2018,retweet,account_category
0,9.060000e+17,10_GOP,"""We have a sitting Democrat US Senator on tria...",Unknown,English,10/1/2017 19:58,10/1/2017 19:59,1052,9636,253,NaN,Right,0,0,RightTroll
1,9.060000e+17,10_GOP,Marshawn Lynch arrives to game in anti-Trump s...,Unknown,English,10/1/2017 22:43,10/1/2017 22:43,1054,9637,254,NaN,Right,0,0,RightTroll
2,9.060000e+17,10_GOP,Daughter of fallen Navy Sailor delivers powerf...,Unknown,English,10/1/2017 22:50,10/1/2017 22:51,1054,9637,255,RETWEET,Right,0,1,RightTroll
3,9.060000e+17,10_GOP,JUST IN: President Trump dedicates Presidents ...,Unknown,English,10/1/2017 23:52,10/1/2017 23:52,1062,9642,256,NaN,Right,0,0,RightTroll
4,9.060000e+17,10_GOP,"19,000 RESPECTING our National Anthem! #StandF...",Unknown,English,10/1/2017 2:13,10/1/2017 2:13,1050,9645,246,RETWEET,Right,0,1,RightTroll


In [6]:
# Process english only tweets

eng_only = tweet_data[tweet_data['language'] == 'English']

eng_only.head()

,external_author_id,author,content,region,language,publish_date,harvested_date,following,followers,updates,post_type,account_type,new_june_2018,retweet,account_category
0,9.060000e+17,10_GOP,"""We have a sitting Democrat US Senator on tria...",Unknown,English,10/1/2017 19:58,10/1/2017 19:59,1052,9636,253,NaN,Right,0,0,RightTroll
1,9.060000e+17,10_GOP,Marshawn Lynch arrives to game in anti-Trump s...,Unknown,English,10/1/2017 22:43,10/1/2017 22:43,1054,9637,254,NaN,Right,0,0,RightTroll
2,9.060000e+17,10_GOP,Daughter of fallen Navy Sailor delivers powerf...,Unknown,English,10/1/2017 22:50,10/1/2017 22:51,1054,9637,255,RETWEET,Right,0,1,RightTroll
3,9.060000e+17,10_GOP,JUST IN: President Trump dedicates Presidents ...,Unknown,English,10/1/2017 23:52,10/1/2017 23:52,1062,9642,256,NaN,Right,0,0,RightTroll
4,9.060000e+17,10_GOP,"19,000 RESPECTING our National Anthem! #StandF...",Unknown,English,10/1/2017 2:13,10/1/2017 2:13,1050,9645,246,RETWEET,Right,0,1,RightTroll


In [7]:
# import re

# Get two months of tweets data
# two_month_tweets = eng_only[eng_only['publish_date'].str.contains('^((6)|(7))\/\d+\/2017', regex='true') ]

one_month_tweets = eng_only[eng_only['publish_date'].str.contains('^6\/\d+\/2017', regex='true') ]

# Used as a pool for TFIDF
tweets_2017 = eng_only[eng_only['publish_date'].str.contains('2017', regex='true') ]

# two_month_tweets.head()
    
one_month_tweets_text = one_month_tweets['content'].astype(str).values.tolist()

tweets_2017_text = tweets_2017['content'].astype(str).values.tolist()

print(len(one_month_tweets_text))

# two_month_tweets_text[0]

35804


In [8]:
import re

import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

from nltk.stem import PorterStemmer

# Get rid of stop words from each tweet
english_stopwords = stopwords.words('english')

# A new array to store words of each tweet after processing
tweet_words = []

# Get the stem of each word
stemmer = PorterStemmer()

for tweet in one_month_tweets_text:

  # Remove @ handle and letters after it
  twit_temp = re.sub(r'@(\w+)', '', tweet.lower())

  # Remove # hashtag but keep letters after it
  twit_temp = re.sub(r'#(\w+)', r'\g<1>', twit_temp)

  # Remove all the links
  twit_temp = re.sub(r'(http|https|ftp)://[a-zA-Z0-9\\./]+', '', twit_temp)

  # Remove all the emoji
  emoji_pattern = re.compile("["
                         u"\U0001F600-\U0001F64F"  # emoticons
                         u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                         u"\U0001F680-\U0001F6FF"  # transport & map symbols
                         u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                         "]+", flags=re.UNICODE)

  twit_temp = re.sub(emoji_pattern, '', twit_temp)

  # Remove all the punctuation
  twit_temp = re.sub(r'[^\w\s]', '', twit_temp)
  twit_temp = twit_temp.split()
  
  twit_clean = []

  # Remove stopwords and stemming the word
  for x in twit_temp:
      if x not in english_stopwords:
          twit_clean.append(stemmer.stem(x))
  
  tweet_words.append(twit_clean)
  
print(tweet_words[0])

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
['putin', 'life', 'putin', 'love']


In [0]:
tweet_words_2017 = []

for tweet in tweets_2017_text:

  # Remove @ handle and letters after it
  twit_temp = re.sub(r'@(\w+)', '', tweet.lower())

  # Remove # hashtag but keep letters after it
  twit_temp = re.sub(r'#(\w+)', r'\g<1>', twit_temp)

  # Remove all the links
  twit_temp = re.sub(r'(http|https|ftp)://[a-zA-Z0-9\\./]+', '', twit_temp)

  # Remove all the emoji
  emoji_pattern = re.compile("["
                         u"\U0001F600-\U0001F64F"  # emoticons
                         u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                         u"\U0001F680-\U0001F6FF"  # transport & map symbols
                         u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                         "]+", flags=re.UNICODE)

  twit_temp = re.sub(emoji_pattern, '', twit_temp)

  # Remove all the punctuation
  twit_temp = re.sub(r'[^\w\s]', '', twit_temp)
  twit_temp = twit_temp.split()
  
  twit_clean = []

  # Remove stopwords and stemming the word
  for x in twit_temp:
      if x not in english_stopwords:
          twit_clean.append(stemmer.stem(x))
  
  tweet_words_2017.append(twit_clean)

In [10]:
# Tweets collection after removing the redundant ones
tweets_org_2017 = []

# Remove the redundant lines
lines_seen_2017 = set()

for line in tweet_words_2017:
    # Ignore the tweet if it has already been processed
    line_tmp = " ".join(line)
    if line_tmp not in lines_seen_2017:
        lines_seen_2017.add(line_tmp)
#         words = line.split(' ')
        # Ignore the tweet has less than 4 words
        if len(line) > 3:
            tweets_org_2017.append(line)
        
len(tweets_org_2017)

563440

In [13]:
import gensim

import numpy as np

# model = gensim.models.Word2Vec.load('./mymodel')

model = gensim.models.KeyedVectors.load_word2vec_format('/content/drive/My Drive/data/russ_twit_model/myModel.txt',binary = False)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [16]:
# To store top 10000 words
word_vec = {}

# To store 10 top related words
words_related_10 = {}

# Choose top 10000 word as keys for our dictionary
for i in range(10000):
    word_vec[model.wv.index2entity[i]] = i

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  


In [17]:

# Remove the redundant lines
lines_seen = set()

# Tweets collection after removing the redundant ones
tweets_org = []

# Tweets vector collection after removing the repeated words
tweets_vec = []

for line in tweet_words:
    # Ignore the tweet if it has already been processed
    line_tmp = " ".join(line)
    if line_tmp not in lines_seen:
        lines_seen.add(line_tmp)
        twit_vec = {}
#         words = line.split(' ')
        # Ignore the tweet has less than 4 words
        if len(line) > 3:
            tweets_org.append(line)
            for word in line:
                if word in model.wv.vocab:
                    # Store the word that has already been inquired
                    if word not in words_related_10:
                        words_related_10[word] = model.most_similar(positive=[word], topn=10)
                    rela_10 = words_related_10[word]
                    # Put the similarity value to the dictionary, if it has been occupied, keep the bigger value
                    for tup in rela_10:
                        if tup[0] in word_vec:
                            if tup[0] not in twit_vec:
                                twit_vec[word_vec[tup[0]]] = tup[1]
                            else:
                                if tup[1] > twit_vec[word_vec[tup[0]]]:
                                    twit_vec[word_vec[tup[0]]] = tup[1]
            tweets_vec.append(str(twit_vec))
            
tweets_vec[0]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


'{142: 0.7760394811630249, 4393: 0.7571162581443787, 263: 0.6587720513343811, 2449: 0.6520659327507019, 6087: 0.6475723385810852, 2564: 0.6425796151161194, 1597: 0.6405378580093384, 9682: 0.6265789866447449, 2850: 0.6172599792480469, 4657: 0.6163877844810486, 3784: 0.5401042699813843, 5865: 0.5608952045440674, 1415: 0.509224534034729, 3568: 0.4952670931816101, 1494: 0.49044090509414673, 4241: 0.4818803369998932, 7132: 0.4790537357330322, 3257: 0.4767795205116272, 248: 0.4754660129547119, 1884: 0.6318841576576233, 429: 0.6063116788864136, 8180: 0.5648703575134277, 9018: 0.5626915693283081, 6027: 0.5541563034057617, 6750: 0.546898603439331, 3190: 0.5308982133865356, 4364: 0.5292786359786987, 1057: 0.5276892185211182}'

In [0]:
# Function to calculate the cosine similarity
def cosine_s(p1_dic, p2_dic):
    comn = 0
    # p1_norm = 0
    # p2_norm = 0
    for key in p2_dic:
        if key in p1_dic:
            comn += p1_dic[key] * p2_dic[key]

    return comn

In [0]:
# Dictionary to store the clustering result
groups = {}

# number = -1

# Total number of tweets' vectors
length = len(tweets_vec)

# To check if the tweet already stored in dictionary
exist_index = {}

# Store the norm of each vector to avoid re-calculation
norm = []

# Store the dictionary interpreted from txt file
dic_list = []

# An array to store 5 tweets vectors in a group
stand = []

In [0]:
# Calculate the norm of the vector
for i in range(length):
    dic_tmp = eval(tweets_vec[i])
    dic_list.append(dic_tmp)
    norm_tmp = 0
    for key in dic_tmp:
        norm_tmp += np.square(dic_tmp[key])
    norm.append(np.sqrt(norm_tmp))

In [0]:
# Compare the cosine similarity between each tweets' vector
for i in range(length):
    if i not in exist_index:
      if(norm[i] != 0):
        if i not in groups:
          groups[i] = [i]
          stand.append(i)
        elif(len(groups[i]) < 5):
          stand = groups[i][:]
        else:
          stand = groups[i][0:5]
        for j in range(i+1, length):
            if j not in exist_index:
                # b is the target tweet vector
                b = dic_list[j]
                # tmp to store the cosine similarity value
                tmp = 0
                if norm[j] != 0:
                  for k in range(len(stand)):
#                     my_key = str(stand[k])+'+'+str(j)
#                     print(my_key)
#                     if my_key not in cosine_cache:
                    cosine_v = cosine_s(dic_list[stand[k]], b) / (norm[stand[k]] * norm[j])
                    tmp = tmp + cosine_v
#                     cosine_cache[my_key] = cosine_v
#                     else:
#                       tmp = tmp + cosine_cache[my_key]
                tmp = tmp / len(stand)
                # Set the threshold to 0.3 this time
                if tmp > 0.3:
                    groups[i].append(j)
                    if j not in groups:
                        groups[j] = [j]
                    groups[j].append(i)
                    if(len(stand) < 5):
                      stand.append(j)
        # If this group has more than 20 members, all those members will not be re-calculated
        if len(groups[i]) > 20:
            for index in groups[i]:
                exist_index[index] = 1
        stand.clear()

In [26]:
group_key_index = []

for key in groups:
    if len(groups[key]) > 10:
        group_key_index.append(str(groups[key]))
        
group_key_index[0]

'[0, 331, 672, 767, 1113, 1231, 1321, 9158, 10223, 10560, 10562, 10563, 10776, 10780, 11511, 11700, 11841, 12001, 14337, 14816, 14817, 14818, 15445, 16845, 16866, 16910, 17567, 19488, 20851, 24452, 24763]'

In [27]:
tweets_org

# The number of clusters
length_2 = len(group_key_index)

total_num = len(tweets_org_2017)

# To store the group numbers that has more than 20 members
indexs = []

# To store the groups
group = []

# To store the words in tweet
group_t = []

# IDF dict
idf_dict = {}

# Temp list
tmp_list = []

# Topic list
my_topic_lists = []

# Use a threshold of 20 to filter the short groups
for i in range(length_2):
    r = eval(group_key_index[i])
    if len(r) > 20:
        indexs.append(i)
# Go through each groups with more than 20 members
for index in indexs:
    print('finished index ' + str(index))
    group = eval(group_key_index[index])
    # group.append(index)

    for i in group:
        words = tweets_org[i]
        for word in words:
            group_t.append(word)

    words_dic_group = {}

    # To calculate the term frequency
    for word in group_t:
        if word not in words_dic_group:
            words_dic_group[word] = 1
        else:
            words_dic_group[word] += 1

    # twit_all.append(list(flat(group_t)))

    # To calculate the IDF for a chosen tweets list
    for key in words_dic_group:
        if key not in idf_dict:
            idf_dict[key] = 1
            for twit in tweets_org_2017:
                if key in twit:
                    idf_dict[key] += 1

    # Calculate TF-IDF
    for key in words_dic_group:
        words_dic_group[key] = words_dic_group[key] * np.log(total_num/idf_dict[key])

    # Rank the words in each group by TF-IDF value, from high to low
    sorted_words = sorted(words_dic_group.items(), key=lambda kv: kv[1], reverse=True)

    # Output 10 words with biggest TF-IDF value
    for i in range(10):
        tmp_list.append(str(sorted_words[i][0]))
        
    my_topic_lists.append(tmp_list[:])


    group.clear()
    group_t.clear()
    words_dic_group.clear()
    tmp_list.clear()
    # twit_all.clear()

finished index 0
finished index 1
finished index 2
finished index 5
finished index 9
finished index 10
finished index 11
finished index 13
finished index 16
finished index 17
finished index 18
finished index 19
finished index 25
finished index 27
finished index 39
finished index 48
finished index 49
finished index 58
finished index 59
finished index 60
finished index 61
finished index 70
finished index 80
finished index 82
finished index 95
finished index 96
finished index 97
finished index 110
finished index 119
finished index 120
finished index 121
finished index 124
finished index 126
finished index 128
finished index 129
finished index 140
finished index 141
finished index 142
finished index 143
finished index 144
finished index 145
finished index 146
finished index 180
finished index 189
finished index 190
finished index 193
finished index 194
finished index 195
finished index 200
finished index 208
finished index 212
finished index 213
finished index 219
finished index 223
finish

In [33]:
my_topic_lists[21:25]

[['uk',
  'elect',
  'may',
  'prime',
  'minist',
  'gambl',
  'parliament',
  'backfir',
  'politician',
  'result'],
 ['cant',
  'across',
  'countri',
  'laugh',
  'spark',
  'stop',
  'ride',
  'stresseat',
  'respectgirl',
  'quadcopter4'],
 ['travel',
  'ban',
  'cant',
  'manspread',
  'effect',
  'sfo',
  'madrid',
  'balloon',
  'ride',
  'stresseat'],
 ['polic',
  'london',
  'say',
  'car',
  'man',
  'attack',
  'vehicl',
  'kill',
  'shoot',
  'shot']]